In [1]:
%cd ../

/scratch/km817/iREC


In [2]:
import numpy as np
import pandas as pd
np.random.seed(0)
# np.random.seed(0)
# !wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx" --no-check-certificate
data = pd.read_excel('ENB2012_data.xlsx', header=0).iloc[:, :10].values

In [3]:
import torch
import pyro
from torch import nn
import pyro.distributions as dist
from pyro.infer import HMC, MCMC, SVI, NUTS, TraceMeanField_ELBO
from pyro import poutine
from sklearn.datasets import load_boston
import numpy as np
import torch.nn.functional as F
from tqdm.notebook import trange
from rec.utils import kl_estimate_with_mc
import matplotlib.pyplot as plt
import torch.distributions as D
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [4]:
from Laplace_GCN_Code.preds.laplace import Laplace
from Laplace_GCN_Code.preds.likelihoods import GaussianLh

In [5]:
import pandas as pd

In [118]:
x_train = np.loadtxt("Laplace_GCN_Code/data/breast_cancer_scale_presplit/breast_cancer_scale_X_tr.csv")
y_train = np.loadtxt("Laplace_GCN_Code/data/breast_cancer_scale_presplit/breast_cancer_scale_y_tr.csv")
x_test = np.loadtxt("Laplace_GCN_Code/data/breast_cancer_scale_presplit/breast_cancer_scale_X_te.csv")
y_test = np.loadtxt("Laplace_GCN_Code/data/breast_cancer_scale_presplit/breast_cancer_scale_y_te.csv")

In [130]:
# np.random.seed(0)
# shuffle_idxs = np.arange(x_.shape[0])
# np.random.shuffle(shuffle_idxs)

# num_train = int(x_.shape[0] * 0.5)
# train_idxs = shuffle_idxs[:num_train]
# test_idxs = shuffle_idxs[:num_train]
# x_train = x_[train_idxs]
# x_m = x_train.mean(0)
# x_s = x_train.std(0)
# x_train = (x_train - x_m) / x_s
# x_test = x_[test_idxs]
# x_test = (x_test - x_m) / x_s
# y_train = y_[train_idxs]
# y_test = y_[test_idxs]

D_in = x_train.shape[-1]
D_out = 1
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train).reshape(-1,1)
x_test= torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test).reshape(-1,1)

In [131]:
def classification_model(x, y=None, weights=None, in_size=1, num_nodes=10, out_size=1, ELBO_BETA=1.):
    # sample vector of weights for regression
    total_weights = (in_size + 1) * num_nodes + (num_nodes + 1) * num_nodes + (num_nodes + 1) * out_size
    # sample params
    with poutine.scale(scale=ELBO_BETA):
        weights = pyro.sample("weights", dist.Normal(torch.zeros(total_weights), 1.).to_event(1))

    idx = 0
    fc1_weights = weights[idx: idx + in_size * num_nodes].reshape(num_nodes, in_size)
    idx += in_size * num_nodes
    fc1_bias = weights[idx: idx + num_nodes].reshape(num_nodes)
    idx += num_nodes

    fc2_weights = weights[idx: idx + num_nodes * num_nodes].reshape(num_nodes, num_nodes)
    idx += num_nodes * num_nodes
    fc2_bias = weights[idx: idx + num_nodes].reshape(num_nodes)
    idx += num_nodes

    fc3_weights = weights[idx: idx + num_nodes * out_size].reshape(out_size, num_nodes)
    idx += num_nodes * out_size
    fc3_bias = weights[idx: idx + out_size].reshape(out_size)
    idx += out_size

    assert idx == total_weights, "Something wrong with number of weights!"

    # compute forward pass
    batch_shape = x.shape[0]
    x = torch.einsum("ij, kj -> ki", fc1_weights, x) + fc1_bias[None].repeat(batch_shape, 1)
    x = torch.relu(x)

    x = torch.einsum("ij, kj -> ki", fc2_weights, x) + fc2_bias[None].repeat(batch_shape, 1)
    x = torch.relu(x)

    x = torch.einsum("ij, kj -> ki", fc3_weights, x) + fc3_bias[None].repeat(batch_shape, 1)
    logits = x
    
    with pyro.plate("data", x.shape[0]):
        obs = pyro.sample("obs", dist.Bernoulli(logits=logits).to_event(1), obs=y)
    
    return logits

In [251]:
def KDE_guide(x, y=None, weight_samples=None, in_size=D_in, num_nodes=10, out_size=1, ELBO_BETA=None):
    total_weights = (in_size + 1) * num_nodes + (num_nodes + 1) * num_nodes + (num_nodes + 1) * out_size
    iso_noise = pyro.param("iso_noise", torch.tensor(1e-5), constraint=dist.constraints.positive)
    assignment = dist.Categorical(probs=torch.ones(weight_samples.shape[0])).sample()

    # sample assigmnent
    with poutine.scale(scale=ELBO_BETA):
        weights = pyro.sample("weights", dist.Normal(weight_samples[assignment], iso_noise).to_event(1))

    idx = 0
    fc1_weights = weights[idx: idx + in_size * num_nodes].reshape(num_nodes, in_size)
    idx += in_size * num_nodes
    fc1_bias = weights[idx: idx + num_nodes].reshape(num_nodes)
    idx += num_nodes

    fc2_weights = weights[idx: idx + num_nodes * num_nodes].reshape(num_nodes, num_nodes)
    idx += num_nodes * num_nodes
    fc2_bias = weights[idx: idx + num_nodes].reshape(num_nodes)
    idx += num_nodes

    fc3_weights = weights[idx: idx + num_nodes * out_size].reshape(out_size, num_nodes)
    idx += num_nodes * out_size
    fc3_bias = weights[idx: idx + out_size].reshape(out_size)
    idx += out_size

    assert idx == total_weights, "Something wrong with number of weights!"

    # compute forward pass
    batch_shape = x.shape[0]
    x = torch.einsum("ij, kj -> ki", fc1_weights, x) + fc1_bias[None].repeat(batch_shape, 1)
    x = torch.relu(x)

    x = torch.einsum("ij, kj -> ki", fc2_weights, x) + fc2_bias[None].repeat(batch_shape, 1)
    x = torch.relu(x)

    x = torch.einsum("ij, kj -> ki", fc3_weights, x) + fc3_bias[None].repeat(batch_shape, 1)
    logits = x

# HMC

In [274]:
pyro.set_rng_seed(0)
ELBO_BETA = 1.
S=0
num_nodes = 3

# run HMC
kernel = HMC(classification_model, step_size=0.001, num_steps=5, target_accept_prob=0.8)
nuts_kernel = NUTS(classification_model, step_size=0.1, target_accept_prob=0.8, max_tree_depth=5)
mcmc = MCMC(kernel, num_samples=2000, warmup_steps=1000, num_chains=1)
mcmc.run(x_train, y_train, ELBO_BETA=ELBO_BETA, num_nodes=num_nodes, in_size=D_in, out_size=D_out)

Sample: 100%|██████████| 3000/3000 [00:05, 588.87it/s, step size=9.23e-02, acc. prob=0.919]


In [275]:
full_samples['weights'].shape

torch.Size([1000, 49])

Warmup:   8%|▊         | 227/3000 [00:20, 22.40it/s, step size=8.06e-02, acc. prob=0.786]

In [276]:
full_samples = mcmc.get_samples(50)
from pyro.infer import Predictive
pred = Predictive(classification_model, full_samples, return_sites=['obs', '_RETURN'])(x_test, None, num_nodes=num_nodes, in_size=D_in, out_size=D_out)

In [277]:
# 1 is high prob

In [278]:
logits = pred['_RETURN']

dist.Bernoulli(logits=logits).log_prob(y_test).mean()

tensor(-0.0956)

In [281]:
flattened_params = full_samples['weights']
prior = dist.MultivariateNormal(loc=torch.zeros_like(flattened_params[0]),
                                covariance_matrix=torch.eye(flattened_params[0].shape[-1]))

# VAR

In [279]:
optimizer = pyro.optim.Adam({"lr": 1e-2})

# train Factored Gaussian approx
from pyro.infer.autoguide import AutoDiagonalNormal
guide = AutoDiagonalNormal(classification_model)
svi = SVI(classification_model, guide, optimizer, loss=TraceMeanField_ELBO())
num_iterations = 20000
pyro.clear_param_store()
pbar = trange(num_iterations)
losses = []
for j in pbar:
    # calculate the loss and take a gradient step
    loss = svi.step(x_train, y_train, ELBO_BETA=ELBO_BETA, num_nodes=num_nodes, in_size=D_in, out_size=D_out)
    losses.append(loss)
    pbar.set_description("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_train)))
guide.requires_grad_(False)

params = []
for name, value in pyro.get_param_store().items():
    params.append(pyro.param(name))

  0%|          | 0/20000 [00:00<?, ?it/s]

In [282]:
means, stds = params
variational_posterior = dist.MultivariateNormal(loc=means, covariance_matrix=torch.diag(stds ** 2))
variational_sample = variational_posterior.sample((50,))
variational_samples = {"weights" : variational_sample}
kl_var_prior = kl_estimate_with_mc(variational_posterior, prior)
var_pred = Predictive(classification_model, variational_samples, return_sites=['obs', '_RETURN'])(x_test, None, 
                                                                        num_nodes=num_nodes, in_size=D_in,
                                                                                             out_size=D_out)


In [283]:
kl_var_prior

tensor(37.0640)

In [284]:
logits = var_pred['_RETURN']

dist.Bernoulli(logits=logits).log_prob(y_test).mean()

tensor(-0.0995)

# KDE

In [285]:
# train KDE
optimizer = pyro.optim.Adam({"lr": 1e-1})
svi = SVI(classification_model, KDE_guide, optimizer, loss=TraceMeanField_ELBO())

num_iterations = 10000
pyro.clear_param_store()
pbar = trange(num_iterations)
losses = []
for j in pbar:
    # calculate the loss and take a gradient step
    loss = svi.step(x_train, y_train, full_samples['weights'], ELBO_BETA=ELBO_BETA, num_nodes=num_nodes, in_size=D_in, out_size=D_out)
    losses.append(loss)
    pbar.set_description("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_train)))
kde_noise = pyro.param("iso_noise")
flattened_params = full_samples['weights']
kde_mix = dist.Categorical(probs=torch.ones(flattened_params.shape[0]))
kde_comps = dist.MultivariateNormal(loc=flattened_params,
                                    covariance_matrix=kde_noise * torch.eye(flattened_params.shape[-1]))
kde = dist.MixtureSameFamily(kde_mix, kde_comps)
prior = dist.MultivariateNormal(loc=torch.ones_like(flattened_params[0]),
                                covariance_matrix=torch.eye(flattened_params[0].shape[-1]))
kl_kde_prior = kl_estimate_with_mc(kde, prior)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [286]:
kde_sample = kde.sample((1000,))
kde_samples = {'weights':kde_sample}
kde_pred = Predictive(classification_model, kde_samples, return_sites=['obs', '_RETURN'])(x_test, None, 
                                                                        num_nodes=num_nodes, in_size=D_in,
                                                                                             out_size=D_out)

In [287]:
logits = kde_pred['_RETURN']

dist.Bernoulli(logits=logits).log_prob(y_test).mean()

tensor(-0.2278)

In [288]:
kl_kde_prior

tensor(100.3488, grad_fn=<MeanBackward1>)

# Test Laplace Approx

In [202]:
from Laplace_GCN_Code.preds.models import MLPS
from Laplace_GCN_Code.preds.likelihoods import BernoulliLh
from Laplace_GCN_Code.preds.laplace import Laplace
model = MLPS(input_size=D_in, hidden_sizes=[num_nodes]*1, output_size=D_out)

In [228]:
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
losses = list()
n_epochs = 50000
prior_prec = 1.
lh = BernoulliLh()
for i in range(n_epochs):
    f = model(x_train)
    w = parameters_to_vector(model.parameters())
    reg = 0.5 * prior_prec * w @ w
    loss = - lh.log_likelihood(y_train, f) + reg
    loss.backward()
    optim.step()
    losses.append(loss.item())
    model.zero_grad()

In [229]:
lap = Laplace(model, float(prior_prec), lh)

In [230]:
train_loader = [(x_train, y_train)]

In [243]:
lap.infer(train_loader, cov_type='kfac')

In [244]:
dist.Bernoulli(probs=lap.predictive_samples_glm(x_test, n_samples=1000)).log_prob(y_test).mean()

tensor(-0.1096)

# Compress weights

In [ ]:
#lets compress some samples
#### sample weights with compression algorithm
from tqdm.notebook import trange
from rec.beamsearch.Coders.Encoder_Empirical import Encoder
from rec.beamsearch.distributions.CodingSampler import CodingSampler
from rec.beamsearch.distributions.EmpiricalMixturePosterior import EmpiricalMixturePosterior
from rec.beamsearch.samplers.GreedySampling_BNNs import GreedySampler
import pyro.distributions as dist

In [ ]:
dummy_model = deterministic_regression_model(full_samples['params'][10], in_size=D_in, num_nodes=num_nodes, out_size=D_out)


In [ ]:
samples = full_samples['params']

In [ ]:
kl_q_p = kl_kde_prior

In [ ]:
from rec.OptimisingVars.FinalJointOptimiser import FinalJointOptimiser
coding_sampler = CodingSampler
auxiliary_posterior = EmpiricalMixturePosterior
selection_sampler = GreedySampler
omega = 5
initial_seed = 0
beamwidth = 1
epsilon = 0.
dummy_encoder = Encoder(dummy_model,
                     x_train[S],
                     y_train[S],
                     samples,
                     initial_seed,
                     coding_sampler,
                     selection_sampler,
                     auxiliary_posterior,
                     omega,
                     beamwidth,
                     epsilon=epsilon,
                     prior_var=1.,
                     total_kl=kl_q_p)

z_sample = samples.mean(0)
omega = 5
n_trajectories = 64
n_auxiliaries = dummy_encoder.n_auxiliary
prior_var = 1.
emp_opt = FinalJointOptimiser(z_sample, omega, n_auxiliaries, kl_q_p, n_trajectories, prior_var)
aux_vars = emp_opt.run_optimiser(epochs=5000)

In [ ]:
del dummy_encoder

In [ ]:
coding_sampler = CodingSampler
auxiliary_posterior = EmpiricalMixturePosterior
selection_sampler = GreedySampler
omega = 5

initial_seed = 0
beamwidth = 1
epsilon = 0.2
num_compressed_samples = 500
compressed_weights_emp_low_eps = []

for i in trange(num_compressed_samples):
    initial_seed = initial_seed + i * 10
    encoder = Encoder(dummy_model,
                     x_train[S],
                     y_train[S],
                     samples,
                     initial_seed,
                     coding_sampler,
                     selection_sampler,
                     auxiliary_posterior,
                     omega,
                     beamwidth,
                     epsilon=epsilon,
                     prior_var=1.,
                     total_kl=kl_q_p)
    
    encoder.auxiliary_posterior.coding_sampler.auxiliary_vars = aux_vars
    
    w, idx = encoder.run_encoder()
    compressed_weights_emp_low_eps.append(w[0])

In [ ]:
weight_samples = torch.zeros([0])
for w in compressed_weights_emp_low_eps:
    weight_samples = torch.cat([weight_samples, w[None]], dim=0)

In [ ]:
weight_samples = {'params':weight_samples}

In [ ]:
compressed = Predictive(regression_model, weight_samples, return_sites=['obs', '_RETURN'])(x_test[S], None, 
                                                                        num_nodes=num_nodes, in_size=D_in,
                                                                                             out_size=D_out)

In [ ]:
compressed_gmm = make_empirical_gmm(weight_samples, num_nodes, x_test[S])

In [ ]:
compressed_gmm.log_prob(y_test[S]).mean()

In [ ]:
from rec.beamsearch.distributions.CodingSampler import CodingSampler
from rec.beamsearch.distributions.KDEPosterior import KDEPosterior
from rec.beamsearch.samplers.GreedySampling_BNNs import GreedySampler
from rec.beamsearch.Coders.Encoder_KDE_BNN import EncoderKDE

In [ ]:
KDE_target = kde

In [ ]:
coding_sampler = CodingSampler
auxiliary_posterior = KDEPosterior
selection_sampler = GreedySampler
omega = 5

initial_seed = 0
beamwidth = 1
epsilon = 0.2
num_compressed_samples = 500
compressed_weights_kde_low_eps = []

for i in trange(num_compressed_samples):
    initial_seed = initial_seed + i * 10
    encoder = EncoderKDE(model=dummy_model,
                         x_data=x_train[S],
                         y_data=y_train[S],
                         target=KDE_target,
                         initial_seed=initial_seed,
                         coding_sampler=coding_sampler,
                         selection_sampler=selection_sampler,
                         auxiliary_posterior=auxiliary_posterior,
                         omega=omega,
                         epsilon=epsilon,
                         beamwidth=beamwidth,
                         prior_var=1.,
                        total_kl=kl_q_p)
    
    encoder.auxiliary_posterior.coding_sampler.auxiliary_vars = aux_vars
    
    w, idx = encoder.run_encoder()
    compressed_weights_kde_low_eps.append(w[0])

In [ ]:
weight_samples_kde = torch.zeros([0])
for w in compressed_weights_kde_low_eps:
    weight_samples_kde = torch.cat([weight_samples_kde, w[None]], dim=0)

In [ ]:
log_probs_kde = torch.zeros([0])
for w in compressed_weights_kde_low_eps:
    dummy_model.make_weights_from_sample(w)
    log_probs_kde = torch.cat([log_probs_kde, dummy_model.data_likelihood(x_train[S], y_train[S])[None]], dim=-1)

In [ ]:
test = {"params":weight_samples_kde[torch.topk(log_probs_kde, k=25)[1]]}

In [ ]:
weight_samples_kde

In [ ]:
weight_samples_kde = {'params':weight_samples_kde}

In [ ]:
compressed = Predictive(regression_model, weight_samples_kde, return_sites=['obs', '_RETURN'])(x_test[S], None, 
                                                                        num_nodes=num_nodes, in_size=D_in,
                                                                                             out_size=D_out)

In [ ]:
compressed_gmm_kde = make_empirical_gmm(weight_samples_kde, num_nodes, x_test[S])

In [ ]:
compressed_gmm_kde.log_prob(y_test[S]).mean()

In [ ]:
compressed_gmm_kde_train = make_empirical_gmm(weight_samples_kde, num_nodes, x_train[S])

compressed_gmm_kde_train.log_prob(y_train[S]).mean()

# Variational Scheme

In [ ]:
#### sample weights with compression algorithm
from rec.beamsearch.Coders.Encoder_Variational import Encoder
from rec.beamsearch.distributions.CodingSampler import CodingSampler
from rec.beamsearch.distributions.VariationalPosterior import VariationalPosterior
from rec.beamsearch.samplers.GreedySampling import GreedySampler
from rec.OptimisingVars.VariationalOptimiser import VariationalOptimiser
from tqdm.notebook import trange
coding_sampler = CodingSampler
auxiliary_posterior = VariationalPosterior
selection_sampler = GreedySampler
omega = 5

initial_seed = 0
beamwidth = 1
epsilon = 0.



compute_params_enc =  encoder = Encoder(variational_posterior,
                      initial_seed,
                      coding_sampler,
                      selection_sampler,
                      auxiliary_posterior,
                      omega,
                      epsilon=epsilon,
                      beamwidth=beamwidth,
                      prior_var=1.)

n_auxiliaries = compute_params_enc.n_auxiliary
kl_q_p = compute_params_enc.total_kl
var_opt = VariationalOptimiser(compute_params_enc.target, omega, n_auxiliaries, kl_q_p, n_trajectories=16, total_var=1.)
aux_vars = var_opt.run_optimiser(epochs=1000, lr=1e-1)

In [ ]:
del compute_params_enc

In [ ]:
initial_seed = 0
beamwidth = 1
epsilon = 0.2

compressed_weights_var_high_eps = []
num_compressed_samples = 500
for i in trange(num_compressed_samples):
    initial_seed = initial_seed + i * 10
    encoder = Encoder(variational_posterior,
                      initial_seed,
                      coding_sampler,
                      selection_sampler,
                      auxiliary_posterior,
                      omega,
                      epsilon=epsilon,
                      beamwidth=beamwidth,
                      prior_var=1.)
    
    encoder.auxiliary_posterior.coding_sampler.auxiliary_vars = aux_vars
    w, idx = encoder.run_encoder()

    compressed_weights_var_high_eps.append(w[0])

In [ ]:
weight_samples_var = torch.zeros([0])
for w in compressed_weights_var_high_eps:
    weight_samples_var = torch.cat([weight_samples_var, w[None]], dim=0)

In [ ]:
weight_samples_var = {'params':weight_samples_var}

In [ ]:
compressed = Predictive(regression_model, weight_samples_var, return_sites=['obs', '_RETURN'])(x_test[S], None, 
                                                                        num_nodes=num_nodes, in_size=D_in,
                                                                                             out_size=D_out)

In [ ]:
compressed_gmm_var = make_empirical_gmm(weight_samples_var, num_nodes, x_test[S])

compressed_gmm_var.log_prob(y_test[S]).mean()

In [ ]:
compressed_gmm_var_train = make_empirical_gmm(weight_samples_var, num_nodes, x_train[S])

compressed_gmm_var_train.log_prob(y_train[S]).mean()

In [ ]:
import pickle as pkl
# pkl.dump(full_samples, open(f"PickledStuff/BNN_UCI/ENERGY/full_samples_split{S}.pkl", "wb"))
# pkl.dump(kde_samples, open(f"PickledStuff/BNN_UCI/ENERGY/kde_samples_split{S}.pkl", "wb"))
pkl.dump(variational_samples, open(f"PickledStuff/BNN_UCI/ENERGY/variational_samples_split{S}.pkl", "wb"))
# pkl.dump(weight_samples_kde, open(f"PickledStuff/BNN_UCI/ENERGY/compressed_kde_samples_split{S}.pkl", "wb"))
# pkl.dump(weight_samples, open(f"PickledStuff/BNN_UCI/ENERGY/compressed_emp_samples_split{S}.pkl", "wb"))
pkl.dump(weight_samples_var, open(f"PickledStuff/BNN_UCI/ENERGY/compressed_var_samples_split{S}.pkl", "wb"))
# pkl.dump(kl_kde_prior, open(f"PickledStuff/BNN_UCI/ENERGY/kde_kl_split{S}.pkl", "wb"))
pkl.dump(kl_var_prior, open(f"PickledStuff/BNN_UCI/ENERGY/var_kl_split{S}.pkl", "wb"))

In [ ]:
print('done')